<a href="https://colab.research.google.com/github/noorehira/ProgrammingForAI/blob/main/morph_vgg16_resnet50_gcn_60_epoch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [5]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

chiragsaipanuganti_morph_path = kagglehub.dataset_download('chiragsaipanuganti/morph')
noorhira_data_deca_path = kagglehub.dataset_download('noorhira/data-deca')

print('Data source import complete.')


Using Colab cache for faster access to the 'morph' dataset.


100%|██████████| 186M/186M [00:10<00:00, 18.7MB/s]

Extracting files...


Data source import complete.


In [6]:
!pip install torch-geometric scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 66.8 MB/s eta 0:00:00


In [7]:
!pip install yacs opencv-python kornia trimesh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 729.1/729.1 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 88.3 MB/s eta 0:00:00


In [8]:
!pip install chumpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chumpy: filename=chumpy-0.70-py3-none-any.whl size=58263 sha256=7b3a77fe5f8a371fa1e481499eb516e2d608c3e4424b7e7234c6babe8c8d5b73
  Stored in directory: /root/.cache/pip/wheels/ae/b7/0e/6f56330e9077b8a6aad99bdb76981b07a7e8b3f056def662a6
Successfully built chumpy


In [9]:
import numpy as np

# Patch deprecated aliases back in (for chumpy)
if not hasattr(np, 'bool'):
    np.bool = bool
if not hasattr(np, 'int'):
    np.int = int
if not hasattr(np, 'float'):
    np.float = float
if not hasattr(np, 'complex'):
    np.complex = complex
if not hasattr(np, 'object'):
    np.object = object
if not hasattr(np, 'str'):
    np.str = str
if not hasattr(np, 'long'):
    np.long = int  # Python 3 only has int
if not hasattr(np, 'unicode'):
    np.unicode = str

/tmp/ipython-input-2599526712.py:12: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, 'object'):
/tmp/ipython-input-2599526712.py:14: FutureWarning: In the future `np.str` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, 'str'):


In [14]:
import shutil
import os

# Create a writable working directory in Colab
writable_deca_path = '/content/DECA'

# Copy DECA folder from the kagglehub download into /content
shutil.copytree(os.path.join(noorhira_data_deca_path, 'DECA'),
                writable_deca_path,
                dirs_exist_ok=True)

# Move into that directory
os.chdir(writable_deca_path)

print("✅ DECA copied and working directory set to:", os.getcwd())


FileNotFoundError: [Errno 2] No such file or directory: 'noorhira/kaggle/input/data-deca/DECA'

In [11]:
deca_file = 'decalib/deca.py'

# Read and patch the file to comment out the renderer setup
with open(deca_file, 'r') as file:
    lines = file.readlines()

# Comment out the renderer setup function
with open(deca_file, 'w') as file:
    for line in lines:
        if 'self._setup_renderer' in line:
            file.write('# ' + line)  # comment it
        else:
            file.write(line)


FileNotFoundError: [Errno 2] No such file or directory: 'decalib/deca.py'

In [12]:
def _setup_renderer(self, model_cfg):
    # Skipping renderer setup due to Kaggle limitations
    self.render = None


In [13]:
import sys
import types
import os

# Set DECA path (for Colab)
deca_path = '/content/DECA'
sys.path.append(deca_path)
os.chdir(deca_path)

# Patch renderer to skip PyTorch3D requirement
from decalib import deca as deca_module

def dummy_renderer(self, model_cfg):
    self.render = None

deca_module.DECA._setup_renderer = dummy_renderer

# Fix inspect compatibility for chumpy
import inspect
if not hasattr(inspect, 'getargspec'):
    inspect.getargspec = inspect.getfullargspec

# Import DECA and config
from decalib.deca import DECA
from decalib.utils.config import cfg as deca_cfg

# Configure DECA
deca_cfg.model.flame_model_path = os.path.join(deca_path, 'data/generic_model.pkl')
deca_cfg.model.use_tex = False  # disable texture to avoid missing .npz
deca_cfg.model.face_mask_path = ''  # optional

deca = DECA(config=deca_cfg)

print("✅ DECA initialized successfully in:", deca_path)


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/DECA'

In [ ]:
def extract_deca_3d_features_batch(img_tensor_batch, deca):
    from torchvision.transforms import Resize, Normalize
    resize = Resize((224, 224))
    normalize = Normalize(mean=[0.5]*3, std=[0.5]*3)

    batch_size = img_tensor_batch.size(0)
    img_batch = torch.stack([resize(img) for img in img_tensor_batch])
    img_batch = normalize(img_batch).to(next(deca.parameters()).device)

    with torch.no_grad():
        codedict = deca.encode(img_batch)
        shape_params = codedict['shape']  # [B, 100]
        exp_params = codedict['exp']      # [B, 50]
        combined = torch.cat([shape_params, exp_params], dim=1)  # [B, 150]

    return combined


In [ ]:
# ==================== MORPH DATASET LOADER (SAVE ANNOTATIONS) ====================
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from pathlib import Path
import re, random

# Custom augmentation
class CustomAugment:
    def __call__(self, img):
        import torchvision.transforms.functional as TF
        import random
        # Flip
        if random.random() > 0.5:
            img = TF.hflip(img)
        # Rotation
        angle = random.uniform(-15, 15)
        img = TF.rotate(img, angle)
        # Color jitter
        color_jitter = transforms.ColorJitter(
            brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1
        )
        img = color_jitter(img)
        # Affine
        scale_val = random.uniform(0.9, 1.1)
        translate_x = random.uniform(-0.05, 0.05) * img.width
        translate_y = random.uniform(-0.05, 0.05) * img.height
        shear_val = random.uniform(-5, 5)
        img = TF.affine(
            img, angle=0,
            translate=(translate_x, translate_y),
            scale=scale_val, shear=shear_val,
            interpolation=TF.InterpolationMode.BILINEAR
        )
        return img

# Custom dataset
class CustomAgeDataset(Dataset):
    def __init__(self, annotations, transform=None, augment=None):
        self.annotations = annotations
        self.transform = transform
        self.augment = augment

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_path, age_label = self.annotations[idx]
        img = Image.open(img_path).convert("RGB")
        if self.augment:
            img = self.augment(img)
        if self.transform:
            img = self.transform(img)
        return img, torch.tensor(age_label, dtype=torch.float32)

# Base preprocessing
base_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])
augment_transform = CustomAugment()

# Function to load MORPH dataset and save annotations
def load_dataset(img_dir, output_file="morph_annotations.txt", max_images=35000):
    img_dir = Path(img_dir)
    annotations = []

    # Match extensions
    for img_path in img_dir.glob("*.*"):
        if img_path.suffix.lower() not in [".jpg", ".jpeg", ".png"]:
            continue

        filename = img_path.stem
        # Extract age from pattern like "M19" or "F43"
        match = re.search(r"[MF](\d+)", filename)
        if match:
            age = int(match.group(1))
            annotations.append((str(img_path), age))
        # Extract age from last part of filename like "00M19" → 19
    suffix = filename.split("_")[-1]  # e.g., "00M19"
    match = re.search(r"(\d+)$", suffix)  # safer: capture ending digits
    if match:
        age = int(match.group(1))
        annotations.append((str(img_path), age))


    if len(annotations) == 0:
        raise ValueError(f"❌ No images found in {img_dir}. Check path or filename pattern.")

    # Shuffle and limit
    random.shuffle(annotations)
    annotations = annotations[:max_images]

    # Save to output_file
    with open(output_file, "w") as f:
        for path, age in annotations:
            f.write(f"{path} {age}\n")

    print(f"✅ Found total images: {len(annotations)}")
    print(f"Sample:", annotations[0])

    # Split train/val
    split_idx = int(0.8 * len(annotations))
    train_annotations = annotations[:split_idx]
    val_annotations = annotations[split_idx:]

    print(f"Train: {len(train_annotations)}, Val: {len(val_annotations)}")

    train_dataset = CustomAgeDataset(train_annotations, transform=base_transform, augment=augment_transform)
    val_dataset = CustomAgeDataset(val_annotations, transform=base_transform, augment=None)

    return train_dataset, val_dataset

# Run loader
img_dir = "/kaggle/input/morph/Dataset/Images/Train"
output_file = "morph_annotations.txt"

train_dataset, val_dataset = load_dataset(img_dir, output_file, max_images=35000)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [ ]:
# Check first 10 annotations directly
with open("morph_annotations.txt", "r") as f:
    lines = f.readlines()[:10]

print("🔍 Sample Annotations:")
for line in lines:
    path, age = line.strip().split()
    print(f"File: {Path(path).name} -> Age: {age}")


In [ ]:
# import re
# from pathlib import Path

# # Base dataset path
# dir_path = Path("/kaggle/input/morph/Dataset/Images/Train")

# images_list, labels_list = [], []

# # Iterate through all jpg/JPG files
# for img_path in dir_path.glob("*.[jJ][pP][gG]"):
#     filename = img_path.stem  # e.g., '262858_00M19'
#     try:
#         # Extract age from last part like '00M19' → '19'
#         suffix = filename.split("_")[-1]  # e.g., '00M19'
#         age_match = re.search(r"(\d+)$", suffix)  # match ending digits
#         if age_match:
#             age = int(age_match.group(1))
#             images_list.append(str(img_path))
#             labels_list.append(age)
#         else:
#             print(f"Skipping {filename} (could not extract age)")
#     except Exception as e:
#         print(f"Error processing {filename}: {e}")

# print(f"Found total images: {len(images_list)}")
# print("First 10 samples:")
# for i in range(min(10, len(images_list))):
#     print((images_list[i], labels_list[i]))


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from torch_geometric.nn import GCNConv
from sklearn.neighbors import NearestNeighbors


In [ ]:
# ==================== MODEL ARCHITECTURE ENHANCEMENTS ====================
def image_to_patches(img_tensor, patch_size=16):
    C, H, W = img_tensor.shape
    patches = []
    coords = []
    h_patches = H // patch_size
    w_patches = W // patch_size
    for i in range(h_patches):
        for j in range(w_patches):
            patch = img_tensor[:, i * patch_size:(i + 1) * patch_size, j * patch_size:(j + 1) * patch_size]
            patches.append(patch)
            coords.append((i, j))
    return patches, coords

def build_knn_graph(features, k=5):
    features = features.cpu().detach().numpy()
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(features)
    _, indices = nbrs.kneighbors(features)
    edges = [(i, j) for i in range(len(indices)) for j in indices[i] if i != j]
    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    return edge_index

def mask_nodes(features, mask_ratio=0.3):
    mask = (torch.rand(features.shape[0], device=features.device) > mask_ratio).float().unsqueeze(1)
    return features * mask  # Apply mask


class AnchorEncoder(nn.Module):
    def __init__(self, output_dim=256):
        super(AnchorEncoder, self).__init__()

        # ResNet-50 backbone
        self.resnet = models.resnet50(pretrained=True)
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-1])
        self.resnet_proj = nn.Sequential(
            nn.Linear(2048, 256), nn.ReLU(), nn.Dropout(0.2)
        )

        # VGG16 backbone
        self.vgg = models.vgg16(pretrained=True).features
        self.vgg_avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.vgg_proj = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 7 * 7, 256),
            nn.ReLU(),
            nn.Dropout(0.2)
        )

        # DECA
        self.deca = deca
        self.deca_proj = nn.Sequential(
            nn.Linear(150, 128),
            nn.ReLU(),
            nn.Dropout(0.1)
        )

        # Final fusion
        self.fusion = nn.Sequential(
            nn.Linear(256 + 256 + 128, output_dim),
            nn.LayerNorm(output_dim),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

    def forward(self, img):
        B = img.size(0)

        # ResNet
        res_feat = torch.flatten(self.resnet(img), 1)
        res_feat = self.resnet_proj(res_feat)

        # VGG16
        vgg_feat = self.vgg(img)
        vgg_feat = self.vgg_avgpool(vgg_feat)
        vgg_feat = self.vgg_proj(vgg_feat)

        # DECA
        with torch.no_grad():
            deca_feat = extract_deca_3d_features_batch(img, self.deca)
        deca_feat = self.deca_proj(deca_feat)

        combined = torch.cat([res_feat, vgg_feat, deca_feat], dim=1)
        return self.fusion(combined)


class RegressionHead(nn.Module):
    def __init__(self, input_dim=256):
        super(RegressionHead, self).__init__()
        self.regressor = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.LayerNorm(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.regressor(x)

class GATLayer(nn.Module):
    def __init__(self, in_features, out_features, dropout=0.2, alpha=0.2):
        super(GATLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.dropout = dropout
        self.alpha = alpha

        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        self.a = nn.Parameter(torch.zeros(size=(2*out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def forward(self, x, edge_index):
        h = torch.mm(x, self.W)
        N = h.size(0)

        a_input = torch.cat([h[edge_index[0]], h[edge_index[1]]], dim=1)
        e = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(1))

        attention = -9e15 * torch.ones(N, N, device=x.device)
        attention[edge_index[0], edge_index[1]] = e
        attention = F.softmax(attention, dim=1)
        attention = F.dropout(attention, self.dropout, training=self.training)

        h_prime = torch.matmul(attention, h)
        return F.elu(h_prime)

class GCNEncoder(nn.Module):
    def __init__(self, in_dim=128, hidden_dim=256, out_dim=256):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, out_dim)
        self.ln1 = nn.LayerNorm(hidden_dim)
        self.ln2 = nn.LayerNorm(out_dim)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x, edge_index):
        x = F.relu(self.ln1(self.conv1(x, edge_index)))
        x = self.dropout(x)
        x = F.relu(self.ln2(self.conv2(x, edge_index)))
        return x

class PatchEncoder(nn.Module):
    def __init__(self, output_dim=128):
        super(PatchEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((2, 2)),
            nn.Flatten(),
            nn.Linear(128 * 2 * 2, output_dim)
        )

    def forward(self, x):
        return self.encoder(x)

class MultipleContrastiveLoss(nn.Module):
    def __init__(self, alpha=0.2, beta=0.3, omega1=1.0, omega2=1.0, omega3=0.5):
        super(MultipleContrastiveLoss, self).__init__()
        self.alpha = alpha
        self.beta = beta
        self.omega1 = omega1
        self.omega2 = omega2
        self.omega3 = omega3

    def forward(self, anchor, positive, negative, neighbor_positive):
        d_ap = F.cosine_similarity(anchor, positive, dim=1)
        d_an = F.cosine_similarity(anchor, negative, dim=1)
        d_np = F.cosine_similarity(anchor, neighbor_positive, dim=1)

        L_N = F.relu(d_an - d_ap + self.alpha).mean()
        L_M = F.relu(d_an - d_np + self.alpha).mean()
        L_V = -(F.relu(d_ap - d_an + self.alpha + self.beta)).mean()

        loss = self.omega1 * L_N + self.omega2 * L_M + self.omega3 * L_V
        return loss


In [ ]:
# ==================== TRAINING UTILITIES ====================
def evaluate(model_anchor, regression_head, dataloader, device, tau=5):
    model_anchor.eval()
    regression_head.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for img, age in dataloader:
            img = img.to(device)
            age = age.cpu().numpy()

            anchor = model_anchor(img)
            predicted_age = regression_head(anchor).squeeze(1).cpu().numpy()

            all_preds.extend(predicted_age)
            all_labels.extend(age)

    mae = mean_absolute_error(all_labels, all_preds)
    abs_errors = np.abs(np.array(all_preds) - np.array(all_labels))
    cs_tau = np.mean(abs_errors <= tau) * 100

    return mae, cs_tau

def train_epoch(model_anchor, patch_encoder, model_gcn, regression_head, optimizer, dataloader, device, contrastive_loss):
    model_anchor.train()
    patch_encoder.train()
    model_gcn.train()
    regression_head.train()
    total_loss = 0.0

    for img, age in dataloader:
        img, age = img.to(device), age.to(device).float().unsqueeze(1)

        anchors = []
        positives = []
        regression_losses = []

        optimizer.zero_grad()

        # Process entire batch at once
        anchor = model_anchor(img)

        # Process each image for graph construction
        for img_tensor, age_val in zip(img, age):
            with torch.no_grad():
                x, edge_index = patch_graph_from_image(img_tensor, patch_encoder, patch_size=32, k=8)
            h_positive = model_gcn(x, edge_index).mean(dim=0, keepdim=True)
            positives.append(h_positive)

        positive = torch.cat(positives, dim=0)
        reg_pred = regression_head(anchor)
        reg_loss = F.smooth_l1_loss(reg_pred, age)

        # Create negative samples by shuffling the batch
        negative = anchor[torch.randperm(anchor.size(0))]

        # Normalize features for contrastive loss
        anchor_norm = F.normalize(anchor, dim=1)
        positive_norm = F.normalize(positive, dim=1)
        negative_norm = F.normalize(negative, dim=1)

        con_loss = contrastive_loss(anchor_norm, positive_norm, negative_norm, positive_norm)
        loss = reg_loss + 0.2 * con_loss  # Reduced contrastive weight

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model_anchor.parameters(), max_norm=1.0)
        torch.nn.utils.clip_grad_norm_(patch_encoder.parameters(), max_norm=1.0)
        torch.nn.utils.clip_grad_norm_(model_gcn.parameters(), max_norm=1.0)
        torch.nn.utils.clip_grad_norm_(regression_head.parameters(), max_norm=1.0)

        optimizer.step()
        total_loss += loss.item() * img.size(0)

    return total_loss / len(dataloader.dataset)

def patch_graph_from_image(img_tensor, patch_encoder, patch_size=32, k=8, deca=None, deca_proj=None):
    patches, coords = image_to_patches(img_tensor, patch_size=patch_size)
    patch_batch = torch.stack(patches).to(img_tensor.device)  # [N, C, H, W]

    with torch.no_grad():
        patch_feats = patch_encoder(patch_batch)  # [N, D]

        # === Add DECA features per image ===
        if deca is not None and deca_proj is not None:
            deca_feat = extract_deca_3d_features_batch(img_tensor.unsqueeze(0), deca).to(img_tensor.device)  # [1, 150]
            deca_feat = deca_proj(deca_feat)  # [1, 128]
            deca_feat = deca_feat.repeat(patch_feats.size(0), 1)  # [N, 128]
            patch_feats = torch.cat([patch_feats, deca_feat], dim=1)  # [N, 128+128 = 256]

    edge_index = build_knn_graph(patch_feats, k=k)
    x_masked = mask_nodes(patch_feats, mask_ratio=0.2)
    return x_masked.to(img_tensor.device), edge_index.to(img_tensor.device)



In [ ]:
!pip install torch

In [ ]:
# ==================== MAIN TRAINING LOOP ====================
import torch                          # ✅ MUST come first
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data as GeometricData
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_absolute_error
from PIL import Image
import os
from pathlib import Path
import re
import math
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR
import torchvision.transforms.functional as TF
import random
import time


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_dataset, val_dataset = load_dataset(img_dir, output_file)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Initialize models
model_anchor = AnchorEncoder(output_dim=256)
model_anchor.deca = deca
model_anchor = model_anchor.to(device)

patch_encoder = PatchEncoder(output_dim=128).to(device)
model_gcn = GCNEncoder(in_dim=256, hidden_dim=256, out_dim=256).to(device)
regression_head = RegressionHead(input_dim=256).to(device)
contrastive_loss = MultipleContrastiveLoss(alpha=0.5, omega1=0.8, omega2=0.8, omega3=0.2)

# Optimizer with weight decay
optimizer = optim.AdamW([
    {'params': model_anchor.parameters(), 'lr': 1e-4},
    {'params': patch_encoder.parameters(), 'lr': 1e-4},
    {'params': model_gcn.parameters(), 'lr': 1e-4},
    {'params': regression_head.parameters(), 'lr': 1e-4}
], weight_decay=1e-4)

# Learning rate schedulers - REMOVED VERBOSE PARAMETER
scheduler_cosine = CosineAnnealingLR(optimizer, T_max=30, eta_min=1e-6)
scheduler_plateau = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10)

# Training parameters
num_epochs = 25
best_mae = float('inf')
patience_counter = 0
patience_limit = 25  # Increased patience
min_lr = 1e-7  # Minimum learning rate
grad_clip = 0.5  # More aggressive gradient clipping

# Custom learning rate scheduler
def adjust_learning_rate(optimizer, epoch, base_lr=1e-4):
    """Adjust learning rate with warmup and cosine decay"""
    # Warmup for first 10 epochs
    if epoch < 10:
        lr = base_lr * (epoch + 1) / 10
    # Cosine decay to min_lr
    else:
        progress = (epoch - 10) / (num_epochs - 10)
        lr = min_lr + 0.5 * (base_lr - min_lr) * (1 + math.cos(math.pi * progress))

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr

# Additional regularization
label_smoothing = 0.1  # For regression
ema_decay = 0.999  # Exponential moving average

# Create EMA model
class EMA:
    def __init__(self, model, decay=0.999):
        self.model = model
        self.decay = decay
        self.shadow = {}
        self.backup = {}

    def register(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()

    def update(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                new_average = (1.0 - self.decay) * param.data + self.decay * self.shadow[name]
                self.shadow[name] = new_average.clone()

    def apply_shadow(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                self.backup[name] = param.data
                param.data = self.shadow[name]

    def restore(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                param.data = self.backup[name]
        self.backup = {}

# Initialize EMA
ema = EMA(model_anchor, decay=ema_decay)
ema.register()

# Enhanced train epoch

def train_epoch(model_anchor, patch_encoder, model_gcn, regression_head,
                optimizer, dataloader, device, contrastive_loss, epoch,
                deca, deca_proj):

    model_anchor.train()
    patch_encoder.train()
    model_gcn.train()
    regression_head.train()
    total_loss = 0.0

    # Update learning rate
    current_lr = adjust_learning_rate(optimizer, epoch)

    for img, age in dataloader:
        img, age = img.to(device), age.to(device).float().unsqueeze(1)

        anchors = []
        positives = []
        regression_losses = []

        optimizer.zero_grad()

        # Process entire batch at once
        anchor = model_anchor(img)

        # Process each image for graph construction
        for img_tensor in img:
            with torch.no_grad():
                x, edge_index = patch_graph_from_image( img_tensor, patch_encoder, patch_size=32, k=8, deca=deca, deca_proj=deca_proj)
            h_positive = model_gcn(x, edge_index).mean(dim=0, keepdim=True)
            positives.append(h_positive)

        positive = torch.cat(positives, dim=0)
        reg_pred = regression_head(anchor)

        # Label smoothing for regression
        reg_loss = F.smooth_l1_loss(reg_pred, age)
        reg_loss = (1 - label_smoothing) * reg_loss + label_smoothing * F.l1_loss(reg_pred, age)

        # Create negative samples by shuffling the batch
        negative = anchor[torch.randperm(anchor.size(0))]

        # Normalize features for contrastive loss
        anchor_norm = F.normalize(anchor, dim=1)
        positive_norm = F.normalize(positive, dim=1)
        negative_norm = F.normalize(negative, dim=1)

        con_loss = contrastive_loss(anchor_norm, positive_norm, negative_norm, positive_norm)
        loss = reg_loss + 0.1 * con_loss  # Reduced contrastive weight

        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model_anchor.parameters(), max_norm=grad_clip)
        torch.nn.utils.clip_grad_norm_(patch_encoder.parameters(), max_norm=grad_clip)
        torch.nn.utils.clip_grad_norm_(model_gcn.parameters(), max_norm=grad_clip)
        torch.nn.utils.clip_grad_norm_(regression_head.parameters(), max_norm=grad_clip)

        optimizer.step()

        # Update EMA model
        ema.update()

        total_loss += loss.item() * img.size(0)

    return total_loss / len(dataloader.dataset), current_lr


# ==================== TRACKERS FOR VISUALIZATION ====================
train_loss_list = []
valid_mae_list = []
valid_cs_list = []
lr_list = []
epoch_times = []

# ==================== MAIN TRAINING LOOP ====================
for epoch in range(num_epochs):
    start_time = time.time()
    train_loss, current_lr = train_epoch(
        model_anchor,
        patch_encoder,
        model_gcn,
        regression_head,
        optimizer,
        train_loader,
        device,
        contrastive_loss,
        epoch,
        deca=model_anchor.deca,
        deca_proj=model_anchor.deca_proj
    )

    # Evaluate with EMA model
    ema.apply_shadow()
    val_mae, val_cs = evaluate(model_anchor, regression_head, val_loader, device, tau=5)
    ema.restore()

    epoch_time = time.time() - start_time

    # Save metrics for visualization
    train_loss_list.append(train_loss)
    valid_mae_list.append(val_mae)
    valid_cs_list.append(val_cs)
    lr_list.append(current_lr)
    epoch_times.append(epoch_time)

    # Console log
    print(f"Epoch {epoch+1}/{num_epochs} | Time: {epoch_time:.1f}s | LR: {current_lr:.2e}")
    print(f"Train Loss: {train_loss:.4f} | Val MAE: {val_mae:.4f} | CS(5): {val_cs:.2f}%")

    # Early stopping and model checkpoint
    if val_mae < best_mae:
        best_mae = val_mae
        patience_counter = 0
        torch.save({
            'model_anchor': model_anchor.state_dict(),
            'patch_encoder': patch_encoder.state_dict(),
            'model_gcn': model_gcn.state_dict(),
            'regression_head': regression_head.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': epoch,
            'val_mae': val_mae,
            'val_cs': val_cs
        }, 'best_model.pth')
        print(f"✅ New best model saved with MAE: {val_mae:.4f}")
    else:
        patience_counter += 1
        if patience_counter >= patience_limit:
            print(f"⏹ Early stopping at epoch {epoch+1}")
            break


# # ==================== MAIN TRAINING LOOP ====================
# for epoch in range(num_epochs):
#     start_time = time.time()
#     train_loss, current_lr = train_epoch(
#         model_anchor,
#         patch_encoder,
#         model_gcn,
#         regression_head,
#         optimizer,
#         train_loader,
#         device,
#         contrastive_loss,
#         epoch,
#         deca=model_anchor.deca,
#         deca_proj=model_anchor.deca_proj
#     )



#     # Evaluate with EMA model
#     ema.apply_shadow()
#     val_mae, val_cs = evaluate(model_anchor, regression_head, val_loader, device, tau=5)
#     ema.restore()

#     epoch_time = time.time() - start_time

#     print(f"Epoch {epoch+1}/{num_epochs} | Time: {epoch_time:.1f}s | LR: {current_lr:.2e}")
#     print(f"Train Loss: {train_loss:.4f} | Val MAE: {val_mae:.4f} | CS(5): {val_cs:.2f}%")

#     # Early stopping and model checkpoint
#     if val_mae < best_mae:
#         best_mae = val_mae
#         patience_counter = 0
#         torch.save({
#             'model_anchor': model_anchor.state_dict(),
#             'patch_encoder': patch_encoder.state_dict(),
#             'model_gcn': model_gcn.state_dict(),
#             'regression_head': regression_head.state_dict(),
#             'optimizer': optimizer.state_dict(),
#             'epoch': epoch,
#             'val_mae': val_mae,
#             'val_cs': val_cs
#         }, 'best_model.pth')
#         print(f"New best model saved with MAE: {val_mae:.4f}")
#     else:
#         patience_counter += 1
#         if patience_counter >= patience_limit:
#             print(f"Early stopping at epoch {epoch+1}")
#             break

# Load best model for final evaluation
try:
    checkpoint = torch.load('best_model.pth', weights_only=False)
except:
    # Handle weight loading issues
    import numpy
    from torch.serialization import safe_globals
    safe_globals.add(numpy.core.multiarray.scalar)
    checkpoint = torch.load('best_model.pth', weights_only=True)

model_anchor.load_state_dict(checkpoint['model_anchor'])
regression_head.load_state_dict(checkpoint['regression_head'])
final_mae, final_cs = evaluate(model_anchor, regression_head, val_loader, device)
print(f"Final Evaluation | MAE: {final_mae:.4f} | CS(5): {final_cs:.2f}%")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

fig, ax = plt.subplots(figsize=(14,7))

# Components (box label, (x,y))
boxes = [
    ("Input Image", (0.02, 0.55)),
    ("DECA 3D\n(Geometry/Expression/Pose)", (0.25, 0.75)),
    ("ResNet-50\n(Global Features)", (0.25, 0.55)),
    ("VGG16\n(Spatial Features)", (0.25, 0.35)),
    ("Anchor Encoder\n(Fusion of ResNet + VGG + DECA)", (0.5, 0.55)),
    ("Patch Encoder\n(Local Patch Features)", (0.5, 0.25)),
    ("Graph Conv Net\n(GCN over Patches)", (0.72, 0.4)),
    ("Fusion + Regression\nHead", (0.88, 0.55)),
    ("Predicted Age", (1.05, 0.55))
]

# Draw boxes
for text, (x,y) in boxes:
    ax.add_patch(mpatches.FancyBboxPatch(
        (x,y), 0.18, 0.15, boxstyle="round,pad=0.02",
        fc="lightblue", ec="black", lw=1.5
    ))
    ax.text(x+0.09, y+0.075, text, ha="center", va="center",
            fontsize=9, weight="bold")

# Arrows between modules
arrow_pairs = [
    (0.20,0.625,0.25,0.625),   # input → DECA
    (0.20,0.625,0.25,0.585),   # input → ResNet
    (0.20,0.625,0.25,0.385),   # input → VGG

    (0.43,0.8,0.50,0.625),     # DECA → AnchorEncoder
    (0.43,0.625,0.50,0.625),   # ResNet → AnchorEncoder
    (0.43,0.385,0.50,0.625),   # VGG → AnchorEncoder

    (0.43,0.385,0.50,0.275),   # VGG also → PatchEncoder
    (0.50,0.325,0.72,0.45),    # PatchEncoder → GCN

    (0.58,0.625,0.88,0.625),   # AnchorEncoder → Fusion+Reg
    (0.78,0.45,0.88,0.625),    # GCN → Fusion+Reg
    (1.06,0.625,1.05,0.625)    # Final → Prediction
]

for x1,y1,x2,y2 in arrow_pairs:
    ax.annotate("", xy=(x2,y2), xytext=(x1,y1),
                arrowprops=dict(arrowstyle="->", lw=2, color="black"))

ax.axis("off")
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Switch to eval mode
model_anchor.eval()
regression_head.eval()

# Pick a few samples from validation set
num_samples = 8
samples = random.sample(range(len(val_dataset)), num_samples)

fig, axes = plt.subplots(2, num_samples//2, figsize=(18, 6))

for i, idx in enumerate(samples):
    img, true_age = val_dataset[idx]
    img_tensor = img.unsqueeze(0).to(device)

    with torch.no_grad():
        anchor = model_anchor(img_tensor)
        pred_age = regression_head(anchor).item()

    # Convert tensor to image
    img_display = TF.to_pil_image(img)

    ax = axes[i // (num_samples//2), i % (num_samples//2)]
    ax.imshow(img_display)
    ax.axis("off")
    ax.set_title(f"True: {true_age:.0f}\nPred: {pred_age:.1f}", fontsize=10)

plt.tight_layout()
plt.show()


In [ ]:
y_true = []
y_pred = []

model_anchor.eval()
regression_head.eval()

with torch.no_grad():
    for img, age in val_loader:
        img = img.to(device)
        age = age.cpu().numpy()

        pred = regression_head(model_anchor(img)).cpu().numpy()

        y_true.extend(age)
        y_pred.extend(pred)

plt.figure(figsize=(7, 7))
plt.scatter(y_true, y_pred, alpha=0.5, edgecolors="k")
plt.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)], "r--", lw=2)
plt.xlabel("True Age", fontsize=12)
plt.ylabel("Predicted Age", fontsize=12)
plt.title("True vs Predicted Ages", fontsize=14)
plt.grid(alpha=0.3)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

# Example: 9 patches → graph
G = nx.grid_2d_graph(3,3)
pos = {(x,y):(x,y) for x,y in G.nodes()}

plt.figure(figsize=(5,5))
nx.draw(G, pos, with_labels=False, node_color="skyblue", node_size=800, edge_color="gray")
plt.title("Patch-level Graph Construction")
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Suppose you collected these during training
train_losses = []   # append(train_loss) inside loop
val_maes = []       # append(val_mae) inside loop
val_cs5 = []        # append(val_cs) inside loop

# Plot
fig, ax1 = plt.subplots(figsize=(8,5))

ax1.plot(train_losses, label="Train Loss", color="blue")
ax1.set_ylabel("Loss", color="blue")
ax1.set_xlabel("Epoch")

ax2 = ax1.twinx()
ax2.plot(val_maes, label="Val MAE", color="red")
ax2.set_ylabel("MAE", color="red")

plt.title("Training Loss & Validation MAE over Epochs")
plt.show()

# CS(5) accuracy curve
plt.figure(figsize=(7,5))
plt.plot(val_cs5, marker="o", color="green")
plt.title("Validation CS(5) across Epochs")
plt.xlabel("Epoch")
plt.ylabel("CS(5) %")
plt.grid(True)
plt.show()
